In [3]:
from ingestion.vector_store.pinecone.client import PineconeClient
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    Event,
    step,
)
import asyncio
from ingestion.utils.pdf_processor import process_pdfs_in_directory
from event import InitializeEvent, PDFIngestionEvent, ConciergeEvent
from ingestion.vector_store.pinecone.client import PineconeClient


class ConciergeWorkflow(Workflow):
    @step(pass_context=True)
    async def concierge(self, ctx: Context, ev: ConciergeEvent | StartEvent) -> InitializeEvent | StopEvent | None:
    
        if isinstance(ev, StartEvent):
            dirname = ev.get("dirname")
            if dirname is not None:
                ctx.data["dirname"] = dirname
                
            return InitializeEvent()
        
        return StopEvent(result={"index": ctx.data["index"]})
    
    @step(pass_context=True)
    async def initialize(self, ctx: Context, ev: InitializeEvent) -> PDFIngestionEvent | ConciergeEvent:
        # Check if "dirname" exists in ctx.data and is not None
        if ctx.data.get("dirname") is not None:
            return PDFIngestionEvent(dirname=ctx.data["dirname"])
        
        ctx.data["index"] = PineconeClient(collection_name="pdf-docs")
        return ConciergeEvent()
    
    @step(pass_context=True)
    async def ingest(self, ctx: Context, ev: PDFIngestionEvent) -> InitializeEvent | None:
        """Entry point to ingest a document, triggered by a StartEvent with `dirname`."""
        dirname = ctx.data.get("dirname")
        if dirname:
            nodes = process_pdfs_in_directory(dirname)

            client = PineconeClient(collection_name="pdf-docs")
            client.upsert_indices(nodes)
            ctx.data["dirname"] = None
            return InitializeEvent()
        return None

In [4]:
concierge = ConciergeWorkflow(timeout=180, verbose=True)
result = await concierge.run()
print(str(result))

Running step concierge
Step concierge produced event InitializeEvent
Running step initialize
Step initialize produced event ConciergeEvent
Running step concierge
Step concierge produced event StopEvent
{'index': <ingestion.vector_store.pinecone.client.PineconeClient object at 0x1074d4510>}


In [5]:
# set Logging to DEBUG for more detailed outputs
query_engine = result["index"].index.as_query_engine()
response = query_engine.query("how does Environmental difficulties at present are also challenging the sustainability of the Mediterranean way of living")

In [6]:
from IPython.display import Markdown, display

display(Markdown(f"{response}"))

Environmental difficulties at present are challenging the sustainability of the Mediterranean way of living due to factors such as water scarcity, land degradation, and declining biodiversity. Water scarcity is a significant issue in Mediterranean-neighboring countries, driven by decreasing water availability and increasing water needs, particularly for agriculture. Land degradation is also a concern, caused by urbanization, pollution, erosion, and other factors, impacting food production. Additionally, global climate changes are leading to water scarcity, land degradation, and crop failures. The decline in biodiversity and the shift towards monocultures further threaten local food production in the Mediterranean regions.